### Transfer Learning example

This notebook based on https://towardsdatascience.com/cnn-transfer-learning-fine-tuning-9f3e7c5806b2



What if we removed the last layer of the VGG16, which simply takes a probability for each of the 1000 classes in the ImageNet and replaces it with a layer that takes 10 probabilities? This way, we could take all the knowledge that VGG16 has trained on the ImageNet and apply it to our problem!
For this, we would initialize our network with the weights from the ImageNet, and then freeze all the convolutional and max-pooling layers so that they do not modify their weights, leaving only the fully connected ones free.
Once this is done, we would start retraining. In this way, we manage to take advantage of the feature extraction stage of our network and only tune the final classifier to work better with our dataset. This is what is known as Transfer Learning because we take advantage of the knowledge of another problem to solve the one we are dealing with.


In [ ]:
# We first load the necessary libraries, the dataset 
import tensorflow as tf
from keras import callbacks
from keras import optimizers
from tensorflow.keras import Model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np


In [ ]:
#Reshape its dimensons to the minimum allowed by the VGG16 --> (48,48,3)
input_shape = (48, 48, 3)
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
# resize train set
X_train_resized = []
for img in X_train:
  X_train_resized.append(np.resize(img, input_shape) / 255)
  
X_train_resized = np.array(X_train_resized)
print(X_train_resized.shape)

(50000, 48, 48, 3)


In [ ]:
# resize test set
X_test_resized = []
for img in X_test:
  X_test_resized.append(np.resize(img, input_shape) / 255)
  
X_test_resized = np.array(X_test_resized)
print(X_test_resized.shape)

(10000, 48, 48, 3)


In [ ]:
# We build the base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                              

In [ ]:
# We freeze every layer in our base model so that they do not train, we want that our feature extractor stays as before --> transfer learning
for layer in base_model.layers: 
  layer.trainable = False
  print('Layer ' + layer.name + ' frozen.')

Layer input_1 frozen.
Layer block1_conv1 frozen.
Layer block1_conv2 frozen.
Layer block1_pool frozen.
Layer block2_conv1 frozen.
Layer block2_conv2 frozen.
Layer block2_pool frozen.
Layer block3_conv1 frozen.
Layer block3_conv2 frozen.
Layer block3_conv3 frozen.
Layer block3_pool frozen.
Layer block4_conv1 frozen.
Layer block4_conv2 frozen.
Layer block4_conv3 frozen.
Layer block4_pool frozen.
Layer block5_conv1 frozen.
Layer block5_conv2 frozen.
Layer block5_conv3 frozen.
Layer block5_pool frozen.


In [ ]:
# We take the last layer of our the model and add it to our classifier
last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)
x = Dense(10, activation='softmax', name='predictions')(x)
model = Model(base_model.input, x)

In [ ]:
# We compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# We start the training
epochs = 10
batch_size = 256
# We train it
model.fit(X_train_resized, Y_train,
          batch_size=batch_size,
          validation_data=(X_test_resized, Y_test),
          epochs=epochs)

Epoch 1/10
196/196 [==============================] - 1341s 7s/step - loss: 1.7858 - accuracy: 0.3581 - val_loss: 1.6672 - val_accuracy: 0.4092
Epoch 2/10
196/196 [==============================] - 1344s 7s/step - loss: 1.6426 - accuracy: 0.4120 - val_loss: 1.6183 - val_accuracy: 0.4182
Epoch 3/10
196/196 [==============================] - 1339s 7s/step - loss: 1.5972 - accuracy: 0.4309 - val_loss: 1.5969 - val_accuracy: 0.4337
Epoch 4/10
196/196 [==============================] - 1338s 7s/step - loss: 1.5703 - accuracy: 0.4390 - val_loss: 1.5658 - val_accuracy: 0.4408
Epoch 5/10
196/196 [==============================] - 1342s 7s/step - loss: 1.5430 - accuracy: 0.4492 - val_loss: 1.5619 - val_accuracy: 0.4411
Epoch 6/10
196/196 [==============================] - 1342s 7s/step - loss: 1.5239 - accuracy: 0.4584 - val_loss: 1.5553 - val_accuracy: 0.4421
Epoch 7/10
196/196 [==============================] - 1343s 7s/step - loss: 1.5049 - accuracy: 0.4630 - val_loss: 1.5355 - val_accuracy:

In [ ]:
# We evaluate the accuracy and the loss in the test set
scores = model.evaluate(X_test_resized, Y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 234s 747ms/step - loss: 1.5113 - accuracy: 0.4568
Test loss: 1.5112800598144531
Test accuracy: 0.45680001378059387
